<a href="https://colab.research.google.com/github/parthasarathi1234/DL_assignment2/blob/main/DL_Assignment_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import DataLoader
import torchvision.transforms as transforms
from torchvision import datasets, transforms
from sklearn.model_selection import train_test_split
from torch.utils.data import DataLoader, SubsetRandomSampler
!pip install wandb
import wandb

In [3]:
transform = transforms.Compose([
    transforms.Resize((128,128)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485,0.456,0.406], std=[0.229, 0.224, 0.225])
])
train_dataset=datasets.ImageFolder(root='inaturalist_12K/train',transform=transform)
test_dataset=datasets.ImageFolder(root='inaturalist_12K/val',transform=transform)

train_indices, val_indices = train_test_split(list(range(len(train_dataset))), test_size=0.2, random_state=42)


train_sampler = SubsetRandomSampler(train_indices)
val_sampler = SubsetRandomSampler(val_indices)

# Creating DataLoader instances for training and validation
train_loader = DataLoader(train_dataset, batch_size=32, sampler=train_sampler)
val_loader = DataLoader(train_dataset, batch_size=32, sampler=val_sampler)
print(len(train_dataset))
print(len(test_dataset))

print(len(train_sampler))
print(len(val_sampler))
print(len(train_loader))
print(len(val_loader))

In [ ]:
class SmallCNN(nn.Module):
    def __init__(self,
                 num_filters,
                 activation,
                 data_augmentation,
                 batch_normalization,
                 dense_neurons,
                 dropout
                ):
        self.activation=activation;
        super(SmallCNN, self).__init__()

        # 1st convolutional layer
        self.conv1 = nn.Conv2d(in_channels=3, out_channels=num_filters, kernel_size=3, padding=1)
        # 2nd convolutional layer
        self.conv2 = nn.Conv2d(num_filters, num_filters * 2, kernel_size=3, padding=1)
        # 3rd convolutional layer
        self.conv3 = nn.Conv2d(num_filters * 2, num_filters * 4, kernel_size=3, padding=1)
        # 4th convolutional layer
        self.conv4 = nn.Conv2d(num_filters * 4, num_filters * 8, kernel_size=3, padding=1)
        # 5th convolutional layer
        self.conv5 = nn.Conv2d(num_filters * 8, num_filters * 16, kernel_size=3, padding=1)
        # max pooling
        self.pool = nn.MaxPool2d(kernel_size=2, stride=2)
        # dense layer
        self.fc1 = nn.Linear(num_filters * 16 * 4*4, dense_neurons)
        # output dense layer
        self.fc2 = nn.Linear(dense_neurons, 10)  # Output layer with 10 neurons for classification

        self.dropout=nn.Dropout(dropout)
        self.batch_norm=nn.BatchNorm2d(num_filters) if batch_normalization else None

    def forward(self, x):
        if(self.activation=='ReLU'):
            x = self.pool(F.relu(self.conv1(x)))
            x = self.pool(F.relu(self.conv2(x)))
            x = self.pool(F.relu(self.conv3(x)))
            x = self.pool(F.relu(self.conv4(x)))
            x = self.pool(F.relu(self.conv5(x)))
        if(self.activation=='sigmoid'):
            x = self.pool(F.sigmoid(self.conv1(x)))
            x = self.pool(F.sigmoid(self.conv2(x)))
            x = self.pool(F.sigmoid(self.conv3(x)))
            x = self.pool(F.sigmoid(self.conv4(x)))
            x = self.pool(F.sigmoid(self.conv5(x)))
        if(self.activation=='tanh'):
            x = self.pool(F.tanh(self.conv1(x)))
            x = self.pool(F.tanh(self.conv2(x)))
            x = self.pool(F.tanh(self.conv3(x)))
            x = self.pool(F.tanh(self.conv4(x)))
            x = self.pool(F.tanh(self.conv5(x)))

        x = x.view(-1, num_filters * 16 * 4*4)

        if(self.activation=='ReLU'):
            x = self.dropout(F.relu(self.fc1(x)))
        if(self.activation=='sigmoid'):
            x = self.dropout(F.sigmoid(self.fc1(x)))
        if(self.activation=='tanh'):
            x = self.dropout(F.tanh(self.fc1(x)))

        x = self.fc2(x)
        return x

    def num_flat_features(self, x):
        size = x.size()[1:]  # all dimensions except the batch dimension
        num_features = 1
        for s in size:
            num_features *= s
        return num_features

# Define hyperparameters



#train network
def train_network():
    with wandb.init() as run:
        config=wandb.config
        cnn_model = SmallCNN(config.num_filters,
                             config.activation,
                             config.data_augmentation,
                             config.batch_normalization,
                             config.dense_neurons,
                             config.dropout
                            ).to(device)

    criterion = nn.CrossEntropyLoss()
    optimizer=optim.Adam(cnn_model.parameters(),lr=config.learning_rate)

    for i in range(config.epochs):
        cnn_model.train()
        train_loss=0.0
        train_correct=0
        train_total=0
        for image,label in train_loader:
            image=image.to(device=device)
            label=label.to(device=device)

            optimizer.zero_grad()
            scores=cnn_model(image)
            loss=criterion(scores,label)

            loss.backward()
            #gradient descent or adam step
            optimizer.step()

            train_loss+=loss.item()
            _,predicted=scores.max(1)
            train_total+=label.size(0)
            train_correct+=predicted.eq(label).sum().item()
        train_loss=train_loss/len(train_loader)
        train_accuracy=100*train_correct/train_total

        num_correct=0
        num_loss=0
        total=0
        cnn_model.eval()

        with torch.no_grad():
            for x,y in val_loader:
                x=x.to(device=device)
                y=y.to(device=device)
                # x=x.reshape(x.shape[0],-1)
                scores=cnn_model(x)
                loss=criterion(scores,y)

                num_loss+=loss.item()
                _,predictions=scores.max(1)
                total+=y.size(0)
                num_correct+=predictions.eq(y).sum().item()
          # print(f'Got {num_correct}/{num_samples} with accuracy {float(num_correct)/float(num_samples)*100:.2f}')
        # cnn_model.train()
        val_accuracy=100*num_correct/total
        val_loss/=len(val_loader)
        wandb.log({"Train_Accuracy" : train_accuracy,"Train_Loss" : train_loss,"Validation_acc" : val_accuracy,"validation_loss" : val_loss,'epoch':i})
        print(f"Train_Accuracy : {train_accuracy},Train_Loss : {train_loss}, Validation_acc : {val_accuracy},validation_loss : {val_loss},epoch:{i}")

# Create the CNN model
device=torch.device('cuda' if torch.cuda.is_available() else 'cpu')


In [ ]:
sweep_config={
    'method':'bayes',
    'name':'Parthu',
    'metric':{
        'goal':'maximize',
        'name':'Validation_acc',
    },
    'parameters':{
        'num_filters':{'values':[32,64]},
        'activation':{'values':['ReLU','sigmoid','tanh']},
        'data_augmentation':{'values':['Yes','No']},
        'batch_normalization':{'values':['Yes','No']},
        'dropout':{'values':[0.2,0.3]},
        'epochs':{'values':[5,10,15]},
        'dense_neurons':{'values':[64,128,256,512]},
        'learning_rate':{'values':[0.0001,0.00001]}
    }
}

sweepId=wandb.sweep(sweep_config,project='cs23m035_DL_Assignment2')
wandb.agent(sweepId,train_network)